# How to query SQLite Cloud database (SQLite3 API)

Before start:

1. Create an account for free on [sqlitecloud.io](https://sqlitecloud.io/).
2. Copy your connection string (keep it secret! It contains your API KEY).

Let's start!

In [ ]:
%pip install sqlitecloud

In [3]:
import sqlitecloud

# You can autoselect the database during the connection by adding the 
# database name as path of the SQLite Cloud connection string, eg:
# conn = sqlitecloud.connect("sqlitecloud://myhost.sqlite.cloud:8860/mydatabase?apikey=myapikey")
sqlitecloud_connection_string = "sqlitecloud://myhost.sqlite.cloud:8860/mydatabase?apikey=myapikey"


In [4]:
# Open the connection to SQLite Cloud
conn = sqlitecloud.connect(sqlitecloud_connection_string)


In [5]:
# Select the database to use if not specified in the connection string
db_name = "chinook.sqlite"
conn.execute(f"USE DATABASE {db_name}")


In [6]:
# The execution of the query generate the `Cursor` object
# to fetch the results.
cursor = conn.execute("SELECT * FROM albums")
cursor.fetchone()


(1, 'For Those About To Rock We Salute You', 1)

In [7]:
# You can use the cursor to perform other queries.
# Queries can be prepared with `question mark` and `named` style
cursor = conn.execute("INSERT INTO genres (Name) values (?)", ("My brand new genre",))

cursor.execute("SELECT * FROM genres WHERE Name like :name", {"name": "My brand%"})

cursor.fetchone()


(31, 'My brand new genre')

In [8]:
# When you are done clean up the connection
conn.close()


### sqlitecloud loves sqlite3

Is your project based on the `sqlite3` library to interact with a SQLite database?

Just install `sqlitecloud` package from `pip` and change the module name! That's it!

Try it yourself:

In [9]:
import sqlite3

import sqlitecloud

# Comment out the following line
conn = sqlite3.connect(":memory:")
# and uncomment this one to use the sqlitecloud package
# conn = sqlitecloud.connect(sqlitecloud_connection_string)

conn.execute(
    "CREATE TABLE IF NOT EXISTS producers (ProducerId INTEGER PRIMARY KEY, name TEXT, year INTEGER)"
)
conn.executemany(
    "INSERT INTO producers (name, year) VALUES (?, ?)",
    [("Sony Music Entertainment", 2020), ("EMI Music Publishing", 2021)],
)

cursor = conn.execute("SELECT * FROM producers")

for row in cursor:
    print(row)


(1, 'Sony Music Entertainment', 2020)
(2, 'EMI Music Publishing', 2021)


### SQLite Cloud for Pandas DataFrame

[Pandas](https://pypi.org/project/pandas/) is a Python package for data manipulation and analysis. It provides high-performance, easy-to-use data structures, such as DataFrame.

Use the connection to SQLite Cloud to:
- Insert data from a DataFrame into a SQLite Cloud database.
- Query SQLite Cloud and fetch the results into a DataFrame for further analysis.

Example:

In [10]:
import io

import pandas as pd

import sqlitecloud

dfprices = pd.read_csv(
    io.StringIO(
        """DATE,CURRENCY,PRICE
    20230504,USD,201.23456
    20230503,USD,12.34567
    20230502,USD,23.45678
    20230501,USD,34.56789"""
    )
)

conn = sqlitecloud.connect(sqlitecloud_connection_string)

conn.executemany("DROP TABLE IF EXISTS ?", [("PRICES",)])

# Write the dataframe to the SQLite Cloud database as a table PRICES
dfprices.to_sql("PRICES", conn, index=False)

# Create the dataframe from the table PRICES on the SQLite Cloud database
df_actual_prices = pd.read_sql("SELECT * FROM PRICES", conn)

# Inspect the dataframe
print(df_actual_prices.head())

       DATE CURRENCY      PRICE
0  20230504      USD  201.23456
1  20230503      USD   12.34567
2  20230502      USD   23.45678
3  20230501      USD   34.56789


In [11]:
# Perform a simple query on the dataframe
query_result = df_actual_prices.query("PRICE > 50.00")

print(query_result)

       DATE CURRENCY      PRICE
0  20230504      USD  201.23456
